This script is only for demo. You can modify the visualizing function to fit your own use.

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
train = pd.read_csv('ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.tail()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
1522060,2024-05-07 23:59:07,359.1,13.4,0,1,0,05-08 05:00,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b
1522064,2024-05-07 23:59:08,336.0,14.3,5,337,0,05-07 23:00,38.98635,-75.13275,62080cff66fc0a8e43c6123a,61d38528b7b7526e1adf3e6f


In [3]:


def visualize_vessel_movements(df):
    """
    Visualize vessel movements on a map with lines and markers for each data point.

    Parameters:
    - df (pandas.DataFrame): A DataFrame with columns ['time', 'latitude', 'longitude', 'vesselId'].

    Returns:
    - A Plotly interactive figure.
    """
    # Ensure 'time' is in datetime format for better tooltip handling
    df['time'] = pd.to_datetime(df['time'])
    
    # Sorting the DataFrame by time to ensure lines are drawn correctly
    df = df.sort_values(by=['vesselId', 'time'])

    # Define a color palette
    color_map = px.colors.qualitative.Plotly

    # Mapping each vessel ID to a color
    unique_vessels = df['vesselId'].unique()
    colors = {vessel_id: color_map[i % len(color_map)] for i, vessel_id in enumerate(unique_vessels)}

    # Create the base map with lines
    fig = px.line_geo(df,
                      lat='latitude',
                      lon='longitude',
                      color='vesselId',
                      color_discrete_map=colors,
                      hover_name='vesselId',
                      hover_data={'time': True, 'latitude': ':.3f', 'longitude': ':.3f'},
                      projection='natural earth',
                      title='Vessel Movements Over Time')

    # Add markers for each data point
    for vessel_id in unique_vessels:
        vessel_data = df[df['vesselId'] == vessel_id]
        fig.add_trace(go.Scattergeo(
            lon=vessel_data['longitude'],
            lat=vessel_data['latitude'],
            mode='markers',
            marker=dict(
                size=8,
                color=colors[vessel_id],
                opacity=0.8,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name=f'Markers for {vessel_id}',
            hoverinfo='text',
            text=vessel_data.apply(lambda row: f'ID: {vessel_id}<br>Time: {row["time"]}<br>Lat: {row["latitude"]:.3f}<br>Lon: {row["longitude"]:.3f}', axis=1)
        ))

    # Enhancing map and layout details
    fig.update_geos(fitbounds="locations", showcountries=True, countrycolor="RebeccaPurple")
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0},
                      coloraxis_colorbar=dict(title="Vessel ID"),
                      title_font_size=20)
    
    return fig

In [ ]:
fig = visualize_vessel_movements(
    train[train['vesselId'] == '61e9f3a8b937134a3c4bfdf7']
)
fig.show()  

In [ ]:
fig = visualize_vessel_movements(
    train[train['vesselId'] == '61e9f3b4b937134a3c4bfe77']
)
fig.show()  

In [ ]:
fig = visualize_vessel_movements(
    train[train['vesselId'] == '61e9f469b937134a3c4c029b']
)
fig.show()  

In [ ]:
i = 2
data = pd.read_csv(f'../data/visual/visualizer_{i}.csv')
fig = visualize_vessel_movements(data)
fig.show()

In [2]:
import pandas as pd
data = pd.read_csv(f'../data/ais_train_kalman.csv')
data.head()


,cog,sog,navstat,latitude,longitude,time_numeric,merged_cluster,time_delta,month,hour,weekday,vessel_embedding,CEU,length,vesselType,latitude_kalman,longitude_kalman,sog_kalman,cog_kalman
0,307.6,17.3,0,7.57302,77.49505,1002660.0,0.0,1393.0,1,14,4,0,6500,199.0,83.0,7.573019,77.495051,17.298020,307.604950
1,306.8,16.9,0,7.65043,77.39404,1004243.0,0.0,1583.0,1,14,4,0,6500,199.0,83.0,7.624642,77.427689,17.033244,307.069205
2,307.9,16.9,0,7.71275,77.31394,1005528.0,0.0,1285.0,1,15,4,0,6500,199.0,83.0,7.679728,77.356573,16.950165,307.588323
3,307.0,16.3,0,7.77191,77.23585,1006787.0,0.0,1259.0,1,15,4,0,6500,199.0,83.0,7.736811,77.281816,16.547891,307.224136
4,307.6,16.1,0,7.81285,77.18147,1007688.0,0.0,901.0,1,15,4,0,6500,199.0,83.0,7.783834,77.219761,16.271176,307.456487


In [ ]:
data_vessel = data[data['vessel_embedding'] == 13]

data_vis_raw = pd.DataFrame({'latitude': data_vessel['latitude'], 'longitude': data_vessel['longitude'], 'vesselId': data_vessel['vessel_embedding'], 'time': data_vessel['time_numeric']})

vessel_embedding_kalman = data_vessel['vessel_embedding'].apply(lambda x: 'kalman_' + str(x))
data_vis_kalman = pd.DataFrame({'latitude': data_vessel['latitude_kalman'], 'longitude': data_vessel['longitude_kalman'], 'vesselId': vessel_embedding_kalman, 'time': data_vessel['time_numeric']})

data_vis = pd.concat([data_vis_raw, data_vis_kalman])

# now time is seconds since newyear 2024, convert to datetime
newyear = pd.to_datetime('2024-01-01')
data_vis['time'] = data_vis['time'].apply(lambda x: newyear + pd.Timedelta(seconds=x))

fig = visualize_vessel_movements(data_vis)
fig.show()

In [6]:
data = pd.read_csv(f'../data/linear_visualization.csv')

fig = visualize_vessel_movements(data)
fig.show()

c:\Users\Simen\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\io\_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.

